In [91]:
%run ./utils.ipynb

✅ Successfully connected to MongoDB.


In [92]:
client = get_mongo_client(uri)

In [93]:
collection = get_collection(client,"pdf_rag_db","chunks")

In [98]:
# Preview a few documents from the MongoDB collection
for doc in collection.find().limit(5):
    print(f"📄 PDF: {doc.get('pdf_name')}, Page: {doc.get('page_number')}, Chunk: {doc.get('chunk_index')}")
    print(f"📝 Text: {doc.get('chunk_text')[:200]}...\n{'-'*80}")

📄 PDF: metformin2.pdf, Page: 1, Chunk: 0
📝 Text: PRODUCT  MONOGRAPH 
 
 
 
 
PrTEVA-METFORMIN 
(metformin hydrochloride) 
 
500 mg and 850 mg Tablets 
 
Teva Standard 
 
 
Oral Antihyperglycemic Agent 
 
 
 
 
Teva Canada Limited  
 
 
             ...
--------------------------------------------------------------------------------
📄 PDF: metformin2.pdf, Page: 2, Chunk: 0
📝 Text: 2
PRODUCT  MONOGRAPH 
 
 
 
TEVA-METFORMIN 
(metformin hydrochloride) 
 
 
500 mg and 850 mg Tablets 
 
 
 
THERAPEUTIC  CLASSIFICATION 
 
Oral Antihyperglycemic Agent 
 
 
ACTIONS  AND  CLINICAL  PHA...
--------------------------------------------------------------------------------
📄 PDF: metformin2.pdf, Page: 2, Chunk: 1
📝 Text: when there is insulin secretion. Metformin, at therapeutic doses, does not cause hypoglycemia 
when used alone in man or in the non-diabetic animal, except when using a near lethal dose.  
Metformin h...
-------------------------------------------------------------------------------

In [95]:
import chromadb
from chromadb.utils import embedding_functions

In [96]:
import chromadb
from chromadb.config import Settings

def store_embedded_chunks_to_chroma(collection):
    """
    Reads chunks from MongoDB collection, embeds them, and stores them in ChromaDB.
    """

    # 1. Init Chroma client (serverless)
    chroma_client = chromadb.Client()

    # 2. Create or get Chroma collection
    chroma_collection = chroma_client.get_or_create_collection(name="pdf_chunks")

    # 3. Prepare data for Chroma
    ids = []
    documents = []
    metadatas = []

    for doc in collection.find():
        # Unique ID per chunk
        chunk_id = f"{doc['pdf_name']}_{doc['page_number']}_{doc['chunk_index']}"

        # Add data
        ids.append(chunk_id)
        documents.append(doc["chunk_text"])
        metadatas.append({
            "pdf_name": doc["pdf_name"],
            "page_number": doc["page_number"],
            "chunk_index": doc["chunk_index"]
        })

    # 4. Add to ChromaDB
    chroma_collection.add(
        ids=ids,
        documents=documents,
        metadatas=metadatas
    )

    print(f"✅ Stored {len(documents)} chunks in ChromaDB.")

In [97]:
# store_embedded_chunks_to_chroma(collection)

In [89]:
view_all_chunks("pdf_chunks")


📦 Found Aaron 344 documents in 'pdf_chunks' collection:

🧩 Chunk 1:
PRODUCT  MONOGRAPH 
 
 
 
 
PrTEVA-METFORMIN 
(metformin hydrochloride) 
 
500 mg and 850 mg Tablets 
 
Teva Standard 
 
 
Oral Antihyperglycemic Agent 
 
 
 
 
Teva Canada Limited  
 
 
                              Date of Preparation: July 25, 2013 
30 Novopharm Court  
 
 
 
 
 
Toronto, Ontario 
 
Canada M1B 2K9 
www.tevacanada.com  
 
Control Number: 165821
🔢 Embedding (first 5 values): [-0.07148081 -0.00065993 -0.11491415 -0.0215324  -0.08020177]
----------------------------------------
🧩 Chunk 2:
2
PRODUCT  MONOGRAPH 
 
 
 
TEVA-METFORMIN 
(metformin hydrochloride) 
 
 
500 mg and 850 mg Tablets 
 
 
 
THERAPEUTIC  CLASSIFICATION 
 
Oral Antihyperglycemic Agent 
 
 
ACTIONS  AND  CLINICAL  PHARMACOLOGY 
 
TEVA-METFORMIN (metformin hydrochloride) is a biguanide derivative producing an 
antihyperglycemic effect which can only be observed in man or in the diabetic animal and only 
when there is insulin secretion.